# Рекомендательная система
В данном проекте мы рассмотрим основные этапы создания рекомендательной системы интернет-магазина.

Основной целью проекта является изучение принципов работы рекомендательных систем, знакомство с различными алгоритмами и создание прототипа.

В ходе работы над проектом были решены следующие задачи:

1) EDA - разведывательный анализ данных

2) Предобработка данных и генерация новых признаков

3) Обучение простой модели LightFM

4) Добавление item_features и user_features

5) Реализация рекомендательной системы с помощью бибилиотеки fast.ai

6) Создание прототипа

# Подготовка данных

In [ ]:
# Загузка библиотек
import numpy as np
import pandas as pd

from collections import Counter
import re

import os

In [ ]:
path = '/module_7'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Загружаем данные
train = pd.read_csv(path + '/data/train.csv')
test = pd.read_csv(path + '/data/test.csv')
submission = pd.read_csv(path + '/data/sample_submission.csv')

"""
# Код для обработки meta данных и подготовки DataFrame
import pandas as pd
import numpy as np
import json

# Список необходимых полей. При необходимости его можно расширить
heads = ['category', 'description', 'title', 'brand', 'also_view', 'main_cat', 'price', 'asin', 'also_buy', 'image', 'rank']

df = pd.DataFrame(columns=heads)

def add_to_df(line, ind):
    """"""Функция, добавляющая строки в датафрейм
    line: - исходная строка
    ind: - индекс датафрейма""""""
    ln = json.loads(line.strip())
    row = []
    for key in heads:
        try:
            row.append(ln[key])
        except:
            row.append(np.nan)
    df.loc[ind] = row
    return 0

# открываем файл
with open('meta_Grocery_and_Gourmet_Food.json') as f:
    ind = 0
    while True:
        # считываем строку
        line = f.readline()
        # прерываем цикл, если строка пустая
        if not line:
            break
        # добавляем строку
        add_to_df(line, ind)
        ind += 1

# закрываем файл
f.close

# Обрабатываем столбец 'rank'
aa = df['rank'].replace('[>#,]','',regex=True).str.strip().str.split(pat='(')
bb = []
cc = []
for x in aa:
    try: 
        bb.append(int(x[0].split('in')[0]))
        cc.append(x[0].split('in')[1].strip())
    except: 
        bb.append(np.nan)
        cc.append(np.nan)

df['rank'] = bb
df['main_rank_cat'] = cc

# Производим замены для одинаковых категорий
df['main_rank_cat'].replace('GroceryGourmetFood', 'Grocery & Gourmet Food', inplace=True)
df['main_rank_cat'].replace(r'Grocery & Gourmet Food.*','Grocery & Gourmet Food', regex=True, inplace=True)
df['main_rank_cat'].replace('Kitchen & D', 'Kitchen & Dining', inplace=True)
df['main_rank_cat'].replace('KitchenD','Kitchen & Dining', regex=True, inplace=True)
df['main_rank_cat'].replace('Home & Kitchen  Kitchen & D','Kitchen & Dining', regex=True, inplace=True)
df['main_rank_cat'].replace(r'Toys & Games.*','Toys & Games', regex=True, inplace=True)
df['main_rank_cat'].replace('ToysGames','Toys & Games', regex=True, inplace=True)
df['main_rank_cat'].replace(r'Patio Lawn & Garden.*','Patio Lawn & Garden', regex=True, inplace=True)
df['main_rank_cat'].replace('ToolsHomeImprovement','Tools & Home Improvement', regex=True, inplace=True)
df['main_rank_cat'].replace('IndustrialScientific','Industrial & Scientific', regex=True, inplace=True)
df['main_rank_cat'].replace('PetSupplies','Pet Supplies', regex=True, inplace=True)
df['main_rank_cat'].replace('HealthHousehold','Health & Household', regex=True, inplace=True)
df['main_rank_cat'].replace('BeautyPersonalCare','Beauty & Personal Care', regex=True, inplace=True)
df['main_rank_cat'].replace('OfficeProducts','Office Products', regex=True, inplace=True)
df['main_rank_cat'].replace('SportsOutdoors','Sports & Outdoors', regex=True, inplace=True)

# Сохраняем результат
df.to_csv(path + '/meta.csv', index=False)
"""
# Полученный файл сохранен в датасете и досупен по адресу: 
# https://www.kaggle.com/segeymakarov/meta-info-for-recommendationsv4

# Загружаем обработанный файл мета данных
meta = pd.read_csv(path + '/data/meta.csv')

In [ ]:
# Удалим дубликаты из мета данных и тренировочного датасета
meta.drop_duplicates(inplace=True)
train.drop_duplicates(inplace = True)

# Объединим тренировочный датасет и данные из meta по идентификатору asin (Amazon Standard Identification Number)
df_new_train = pd.merge(train, meta, on='asin')

Посмотрим на тестовый и тренировочный датасеты

In [ ]:
train.info()
test.info()

Видим, что в тренировочном датасете на три столбца больше:

присутствует overall - оценка по пятибальной шкале
присутствует reviewText - текст отзыва
присутствует summary - краткое содержание отзыва
присутствует rating - целевая переманная
отсутствует Id

In [ ]:
# Объединим тестовый и тренировочный датасеты
train['is_test']=0
test['is_test']=1
big_df = pd.concat([train, test], ignore_index=True)

# Объединим полученный датасет и данные из meta по идентификатору asin (Amazon Standard Identification Number)
df_new_train = pd.merge(big_df, meta, on='asin')
# и создадим его копию
df = df_new_train.copy()

# EDA

Смотрим датасет

In [ ]:
df.head()

In [ ]:
df.info()

Пройдемся по признакам.

Напишем функции, которые дадут необходимую информацию для первичного анализа признака.

In [ ]:
# Функция с гистограммой
def col_info_hist(ys):
    print('Количество пропусков: {},'.format(ys.isnull().sum()))
    print('{},'.format(ys.describe()))
    print('Распределение:\n{},'.format(ys.value_counts()))
    ys.hist()

# Функция без гистограммы


def col_info(ys):
    print('Количество пропусков: {},'.format(ys.isnull().sum()))
    print('{},'.format(ys.describe()))
    print('Распределение:\n{},'.format(ys.value_counts()))

# Функция распределения значений по 3-м категориям


def make_3_cat(x, [low, med], [name_low, name_med, name_high]):
    if x < low:
        x = name_low
    elif x < med:
        x = name_med
    else:
        x = name_high
    return x

# Функция, которая оставляет только топовые категории в списке, остальные удалит


def leave_top_elem(elems, top_elem):
    top_elems = []
    for elem in elems:
        if elem in top_elem:
            top_elems.append(elem)
    x = top_elems
    return x

# Функция для отображения категорий в записи


def find_item(cell):
    if item in cell:
        return 1
    return 0

## Overall

In [ ]:
col_info_hist(df.overall)

Похоже на оценку по пятибальной шкале. Подавляющее большинство - пятерки. Данного признака нет в тестовой выборке, но мы можем использовать его для анализа других признаков.

In [ ]:
df['overall']=df['overall'].fillna(0).astype('int32')

## verified

In [ ]:
col_info(df.verified)

Данный признак скорее всего означает, что либо комментарий, либо аккаунт пользователя подтвержден. Пропусков нет. Заменим значения на 1 и 0.

In [ ]:
dic_verified = {True: 1, False: 0}
df['verified'] = df['verified'].map(dic_verified)

In [ ]:
# Посмотрим, как влияет признак verified на среднюю оценку.

print('Средняя оценка при значении verified = 1: ',df[df.verified == 1].overall.mean())
df['overall'][df['verified'] == 1].hist(bins=10)
print('Средняя оценка при значении verified = 0: ',df[df.verified == 0].overall.mean())
df['overall'][df['verified'] == 0].hist(bins=10)

Видим, что средняя оценка товаров с признаком verified = 1 выше, но в целом распределение оценок похожее.

## reviewTime

Дата отзыва. Эта информация есть в колонке unixReviewTime в более удобном формате. Удалим данный столбец.

In [ ]:
df = df.drop(['reviewTime'], axis=1)

## reviewerName

In [ ]:
col_info(df.reviewerName)

Видим, что довольно много отзывов с Амазона и Киндла. 164 пропуска. 

Проверим, один userid у Amazon Customer, или одному имени пользователя может соответствовать несколько userid.

In [ ]:
df[df.reviewerName=='Amazon Customer'].userid.value_counts()

В таком случае, имя пользователя нам не нужно. Удалим столбец.

In [ ]:
df = df.drop(['reviewerName'], axis=1)

## reviewText

In [ ]:
# данного признака нет в тестовой выборке. Удалим столбец
df = df.drop(['reviewText'], axis=1)

## summary

In [ ]:
#Данного признака нет в тестовом датасете. Удалим столбец.
df = df.drop(['summary'], axis=1)

## asin


идентификатор для модели нам не нужен, но у нас есть признаки also_buy и similar_item, для которых он может понадобиться. Пока оставим.

In [ ]:
col_info(df.asin)

Видим, что всего у нас 41320 различных продукта.

## unixReviewTime

In [ ]:
col_info(df.unixReviewTime)

In [ ]:
# Пропусков нет. Посмотрим, как изменялись оценки со временем.

df.unixReviewTime.hist(bins=100)
df[df.overall == 5].unixReviewTime.hist(bins=100)
df[df.overall == 4].unixReviewTime.hist(bins=100)
df[df.overall == 3].unixReviewTime.hist(bins=100)
df[df.overall == 1].unixReviewTime.hist(bins=100)
df[df.overall == 2].unixReviewTime.hist(bins=100)

In [ ]:
# Посмотрим, на распределение отзывов по датам, приведя даты к удобному виду.

from datetime import datetime
tsmin = df.unixReviewTime.min()
ts25 = int(df.unixReviewTime.quantile(0.25))
ts50 = int(df.unixReviewTime.quantile(0.50))
ts75 = int(df.unixReviewTime.quantile(0.75))
tsmax = df.unixReviewTime.max()
print('Самый первый отзыв:', datetime.utcfromtimestamp(tsmin).strftime('%Y-%m-%d %H:%M:%S'))
print('25 квантиль:', datetime.utcfromtimestamp(ts25).strftime('%Y-%m-%d %H:%M:%S'))
print('50 квантиль:', datetime.utcfromtimestamp(ts50).strftime('%Y-%m-%d %H:%M:%S'))
print('75 квантиль:', datetime.utcfromtimestamp(ts75).strftime('%Y-%m-%d %H:%M:%S'))
print('Последний отзыв:', datetime.utcfromtimestamp(tsmax).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# Заменим значения в столбце на категории
df['unixReviewTime'] = df['unixReviewTime'].apply(lambda x: make_3_cat(x, [ts25, ts75], ['old','middle','new']))

Создание dummy-переменных сделаем позже.

## vote

In [ ]:
col_info(df.vote)

In [ ]:
df.vote.unique()

Скорее всего, данные признак показывает количество голосов, отданных за отзыв. Видим, что в значениях числа через запятую, с нулём и без нуля. Также видим большое количество пропусков. Скорее всего, за эти отзывы никто не проголосовал, что вполне естественно, и мы можем заменить пропуски на 0.

In [ ]:
# Заменим пропуски на 0
df.vote = df.vote.fillna(0)

# Удалим запятые
df['vote'] = df['vote'].astype('str')
df['vote'] = df['vote'].apply(lambda x: x.replace(',', ''))

# Заменим пропуски на 0
df['vote'] = df['vote'].apply(lambda x: x.replace('nan', '0'))
df.vote = df.vote.fillna(0)

# Приведем к int32
df['vote'] = df['vote'].astype('float')
df['vote'] = df['vote'].astype('int32')

# Посмотрим на результат
df.vote.unique()
col_info(df.vote)

In [ ]:
# Заменим значения в столбце на категории
df['vote'] = df['vote'].apply(lambda x: make_3_cat(x, [10,100], ['low','middle','high']))

dummy-переменные создадим после EDA

## style

In [ ]:
col_info(df['style'])

Видим словарь из размера упаковки и вкуса. Скорее всего, отсюда можно что-то полезное выделить, но но пропусков больше половины. Займемся если останется время. Пока удаляем.

In [ ]:
df = df.drop(['style'], axis=1)

## image_x, image_y

Ссылки на картинки. Сделаем столбец с указанием наличия изображения is_image

In [ ]:
def has_image(a):
    if len(a)>2:
        return 1
    return 0

df['is_image'] = df['image_x'].fillna('0') + df['image_y'].fillna('0')
df['is_image'] = df['is_image'].apply(has_image)

In [ ]:
# исходные столбцы image_x, image_y удалим
df = df.drop(['image_x','image_y'], axis=1)

## userid

In [ ]:
col_info(df.userid)

ID пользователя. Пропусков нет, идем дальше.

## itemid

In [ ]:
col_info(df.itemid)

Проверим, если в тестовом и тренировочном датасетах пропущенные id. Это может быть проблемой при предсказании с помощьюу LightFM c item_features.

In [ ]:
print('Количество уникальных itemid в тренировочном датасете:',len(df.itemid.unique()))
print('Максимальный itemid в тренировочном датасете:',df.itemid.max()+1)

Видим, что есть пропущенные itemid. Подумаем, что с этим делать после eda.

In [ ]:
df.head(1)

## rating

In [ ]:
col_info(df.rating)

Целевая переменная. Здесь всего 2 значения - понравился пользователю товар или нет. 

Посмотрим, как этот признак зависит от оценки.

In [ ]:
print('Оценки при rating = 0:\n', df[df.rating == 0].overall.value_counts())
print('Оценки при rating = 1:\n', df[df.rating == 1].overall.value_counts())

При оценках 4 или 5 считаем, что товар понравился. 1,2 или 3 - не понравился.

## category

Видим в данном признаке список категорий, к которй относится товар. Посмотрим, сколько всего категорий и выделим самые популярные.

In [ ]:
# Зададим переменной количество категорий
N_CATS = 50

# Создаем пустой список, в который будут добавляться все категории
all_categories = []

# Добавляем категории каждой записи в общий список
for category in df.category:
    all_categories.extend(category.replace('[','').replace(']','').replace("'",'').split(','))

# Считаем частоту категорий в датасете
cnt = Counter()
for word in all_categories:
    cnt[word] +=1

#Оставим топ N_CATS категорий
top_cat = []
for i in range (0, len(cnt.most_common(N_CATS))):
    cat = cnt.most_common(N_CATS)[i][0]
    top_cat.append(cat)
    
# Удаляем дубликаты из all_categories
all_categories = list(dict.fromkeys(all_categories))

print('Всего категорий: ', len(all_categories))
print('Топ', N_CATS, 'категорий: ',top_cat)

Далее мы можем оставить только самые популярные категории и создать на их основе dummy-переменные.

In [ ]:
top_cat = ['Grocery & Gourmet Food', ' Coffee', ' Beverages', ' Tea & Cocoa', ' Cooking & Baking', ' Snack Foods',
           ' Candy & Chocolate', ' Tea', ' Single-Serve Capsules & Pods', ' Herbs', ' Spices & Seasonings',
           ' Bottled Beverages', ' Water & Drink Mixes', ' Nuts & Seeds', ' Breakfast Foods', ' Canned',
           ' Jarred & Packaged Foods', ' Single Herbs & Spices', ' Sauces', ' Bars', ' Syrups',
           ' Sugars & Sweeteners', ' Cereals', ' Cooking Oils', ' Vinegars & Sprays', ' Produce',
           ' Green', ' Jelly Beans & Gummy Candy', ' Grains & Rice', ' Breakfast & Cereal Bars', ' Jams',
           ' Cookies', ' Soups', ' Stocks & Broths', ' Oils', ' Nut Bars', ' Gravies & Marinades',
           ' Candy & Chocolate Bars', ' Jellies & Sweet Spreads', ' Hard Candy & Lollipops', ' Roasted Coffee Beans',
           ' Packaged Meals & Side Dishes', ' Baking Mixes', ' Tea Samplers', ' Chips & Crisps', ' Mixed Spices & Seasonings',
           ' Ground Coffee', ' Sugar Substitutes', ' Juices', ' Dried Fruits & Vegetables']

# Применим функцию к датасету, оставим только топовые категории в стольце category
df['category'] = df['category'].apply(lambda x: leave_top_elem(x, top_cat))


# Создание dummy-переменных на основе топовых категорий

# Cоздаем столбцы с категориями и заполняем 0 или 1
for item in top_cat:
    column_name = 'category' + item
    df[column_name] = df['category'].apply(find_item)

In [ ]:
# Удалим столбец с категориями
df = df.drop(['category'], axis = 1)

## description

In [ ]:
df.description[115]

найдем самые часто встречающиеся слова, оставим только их и подготовимся к созданию dummy-переменных

In [ ]:
# Зададим переменной количество наиболее частов встречающихся слов, которое хотим оставить
N_WORDS = 200

# Сейчас в поле description список строк. Приведем к единой строке.
df['description']=df['description'].astype('str')

# Разбиваем description на список слов, предварительно приводим текст к нижнему регистру
df['description']=df['description'].apply(lambda x: re.sub("[^\w]", " ",  x.lower()).split())
                                        
# Создаем пустой список, в который будут добавляться все слова
all_words = []

# Добавляем слова каждой записи в общий список
for words in df['description']:
    # разбиваем текст на слова, предварительно приводим к нижнему регистру
    all_words.extend(words)

# Считаем частоту слов в датасете
cnt = Counter()
for word in all_words:
    cnt[word] +=1
    
#Оставим топ N_WORDS слов
top_words = []
for i in range (0, len(cnt.most_common(N_WORDS))):
    words = cnt.most_common(N_WORDS)[i][0]
    top_words.append(words)
    
# Удаляем дубликаты из all_words
all_words = list(dict.fromkeys(all_words))

print('Всего слов ', len(all_words))
print('Топ', N_WORDS, 'слов: ',top_words)

Среди самых частых слов много предлогов и артиклей. Отберем вручную список из 50 значимых по нашему мнению слов.

In [ ]:
top_word_list = ['health','treat','intended','disease','prevent','cure',
                 'dietary','diagnose','evaluated','statements','condition',
                 'supplements','fda','organic','flavor','tea','natural',
                 'sugar','salt','water','oil','coffee','ingredients','milk',
                 'free','products','chocolate','taste','cup','delicious',
                 'quality','flavors','soy','food','flour','acid','wheat',
                 'great','powder','corn','nuts','rice','protein','coconut',
                 'gluten','butter','syrup','blend','high','best']

Далее мы можем оставить только самые популярные слова и создать на их основе dummy-переменные.

In [ ]:
# Применим функцию к датасету, оставим только слова из top_word_list в столбце description
df['description'] = df['description'].fillna('').apply(
    lambda x: leave_top_elem(x, top_word_list))

# Создание dummy-переменных на основе слов из top_word_list

# Cоздаем столбцы со словами из top_word_list и заполняем 0 или 1
for item in top_word_list:
    column_name = 'description' + item
    df[column_name] = df['description'].apply(find_item)

In [ ]:
# Удалим столбец
df = df.drop(['description'], axis = 1)

## title

In [ ]:
df.title[115]

In [ ]:
# Видим, что title частично содержится в description. Удалим данный признак.
df = df.drop(['title'], axis = 1)

## brand

In [ ]:
col_info(df.brand)

Пропусков относительно не много. 8866 разных производителей. Можно выделить самых известных и сделать для них dummy-переменные, как мы делали для других категориальных признаков.

In [ ]:
# Зададим переменной количество производителей
N_BRANDS = 50

# Приведем к str
df['brand']=df['brand'].astype('str')

# Создаем пустой список, в который будут добавляться все производители
all_brands = []

# Добавляем производителя из каждой записи в общий список
for brand in df.brand:
    all_brands.append(brand)

# Считаем количество производителей в датасете
cnt = Counter()
for word in all_brands:
    cnt[word] +=1
    
#Оставим топ N_BRANDS производителей
top_brand = []
for i in range (0, len(cnt.most_common(N_BRANDS))):
    br = cnt.most_common(N_BRANDS)[i][0]
    top_brand.append(br)
    
# Удаляем дубликаты из all_brands
all_brands = list(dict.fromkeys(all_brands))

print('Всего производителей: ', len(all_brands))
print('Топ', N_BRANDS, 'производителей: ',top_brand)

Далее мы можем оставить только самых популярных производителей и создать на их основе dummy-переменные. Перед работой с топовыми производителями нужно будет поработать с пропусками, либо использовать nan как отдельную категорию

In [ ]:
# Применим функцию к датасету, оставим только топовых производителей в стольце brand
df['brand'] = df['brand'].apply(lambda x: leave_top_elem(x, top_word_list))

# Создание dummy-переменных на основе топовых производителей

# Cоздаем столбцы с производителями и заполняем 0 или 1
for item in top_brand:
    column_name = 'brand_' + item
    df[column_name] = df['brand'].apply(find_item)

In [ ]:
# Удаляем исходный столбец
df = df.drop(['brand'], axis = 1)

## rank

In [ ]:
print('Количество пропусков: {},'.format(df['rank'].isnull().sum()))

In [ ]:
# Заменим пропуски на высокий ранг (предположим, что чем ниже ранг, тем лучше)
df['rank'] = df['rank'].fillna(9999999)
df['rank'] = df['rank'].astype('float')

# Остаются какие-то большие значения, заменим их на высокий ранг
df['rank'] = df['rank'].apply(lambda x: 9999999 if x > 9999999 else x)
df['rank'] = df['rank'].astype('int32')

# Посмотрим на результат
col_info(df['rank'])

In [ ]:
# Заменим значения в столбце на категории
df['rank'] = df['rank'].apply(lambda x: make_3_cat(x, [10454,169904], ['high','middle','low']))

## also_view

In [ ]:
df.also_view[127]

Видим список других товаров в виде asin.

В качестве идеи:

- так же как мы делали для категорий, посчитать количество упоминаний каждого товара
- сделать числовой признак in_also_view, в котором будет указано, сколько каждый товар упоминается в других товарах в also_view
- сделать как с категориями - dummy-переменные для само часто упоминаемых в also_view товарах

Пока удалим признак.

In [ ]:
df = df.drop(['also_view'], axis = 1)

## main_cat

In [ ]:
col_info(df.main_cat)

In [ ]:
# Избавимся от пропусков. dummy-переменные для категорий создадим после EDA.

# Заменим пропуски на категорию "Other"
df.main_cat = df.main_cat.fillna('Other')

## price

In [ ]:
col_info(df.price)

У нас много пропусков и есть диапозоны цен, например 19.99-295.00. Заменим такие значения и пропуски на среднее значение.

In [ ]:
# Приведем все к str
df['price'] = df['price'].astype('str')

# Заменим значения диапазонов на минимальное


def repl(st):
    return st.split(' ')[0].split('-')[0].replace('$', '')


df['price'] = df['price'].apply(repl)

# Приведем к float
df['price'] = df['price'].astype('float')

# Заменим пропуски на среднее значение
df['price'] = df['price'].fillna(round(df.price.mean(), 2))

# Смотрим на результат
col_info(df.price)

In [ ]:
# Заменим значения в столбце на категории
df['price'] = df['price'].apply(lambda x: make_3_cat(x, [13.99,21.5], ['low','middle','high']))

## also_buy

In [ ]:
print(df.also_buy[25])

аналогично also_view. Есть идеи, что с этим сделать, но пора переходить к следующим этапам. Пока что удалим.

In [ ]:
df = df.drop(['also_buy'], axis = 1)

## main_rank_cat

In [ ]:
col_info(df.main_rank_cat)

Во многом повторяет поле main_cat. пока удалаяем

In [ ]:
df = df.drop(['main_rank_cat'], axis = 1)

## Id

In [ ]:
# удалим столбец
df = df.drop(['Id'], axis = 1)

## Посмотрим на результат

In [ ]:
df.info()

Мы видим, что осталось обработать 10 категориальных признаков, из которых

3 шт. (userid, itemid, rating) мы не трогаем
признак overall отсутствует в тестовом датасете
а для остальных 6 (verified, unixReviewTime, vote, rank, main_cat, price) можем сделать dummy-переменные.

## Анализ категориальных признаков

Все признаки у нас категориальные и использовать корреляционный анализ не получится. Однако можно посмотреть, различаются ли распределения рейтинга в зависимости от значения этих переменных. Это можно сделать, например, с помощью box-plot или KDE-plot (график показывает плотность распределения переменных).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

In [ ]:
def get_boxplot(column):
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column, y='overall',
                data=df.loc[df.loc[:, column].isin(df.loc[:, column].value_counts().index[:10])], ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()


def get_stat_dif(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'overall'],
                     df.loc[df.loc[:, column] == comb[1], 'overall']).pvalue \
                <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [ ]:
for col in ['verified', 'unixReviewTime', 'vote', 'rank', 'main_cat', 'price']:
    get_boxplot(col)

По графикам сильных различий не видно. Однако графики являются лишь вспомогательным инструментом, настоящую значимость различий может помочь распознать статистика. 

Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента. 

Проверим нулевую гипотезу о том, что распределения оценок пользователей по различным параметрам неразличимы:

In [ ]:
for col in ['verified', 'unixReviewTime', 'vote', 'rank', 'main_cat', 'price']:
    get_stat_dif(col)

Как мы видим, распределение оценок серьезно различается по всем шести параметрам.

In [ ]:
# Удалим признак overall
df = df.drop('overall', axis=1)

## Создадим dummy-переменные

In [ ]:
for column in ['unixReviewTime', 'vote', 'rank', 'main_cat', 'price']:
    dummies = pd.get_dummies(df[column], prefix = df[column].name)
    # Удаляем исходный столбец и добавляем dummies
    df = df.drop(df[column].name, axis=1).join(dummies)

In [ ]:
# Сохраним полученный датафрейм для последующего анализа
df.to_csv(path + '/data/extended_df.csv', index=False)

In [ ]:
del df

# Итоги EDA

Кратко напишем, что мы сделали с каждым признаком:

- overall - оценки от 1 до 5. Нет в тестовой выборке, использовали как вспомогательный признак для анализа.
- verified - привели значения к 0 и 1
- reviewTime - удалили признак т.к. есть unixReviewTime
- asin - удалили признак т.к. не использовали, хотя наверное могли бы для also_buy, also_view, similar_item
- reviewerName - удалили т.к. есть userid
- reviewText - удалили, т.к. признака нет в тестовой выборке
- summary - удалили т.к. нет в тестовом датасете
- unixReviewTime - сделали 4 категории в зависимости от даты отзыва, сделали dummy-переменные
- vote - сделали 3 категории в зависимости от количества голосов, сделали dummy-переменные
- style - удалили т.к. больше половины пропусков
- image_x, image_y - удалили, заменив на поле is_image
- userid - оставили для модели
- itemid - оставили для модели
- rating - оставили для модели - целевая переменная
- category - выдилили самые часто встречающиеся категории, сделали (все подготовили и закомментировали для скорости) dummy-переменные
- description - выдилили самые часто встречающиеся слова, сделали (все подготовили и закомментировали для скорости) dummy-переменные
- title - удалили т.к. сильно похож на description
- brand - выдилили самых часто встречающихся производителей, сделали (все подготовили и закомментировали для скорости) dummy-переменные
- rank - сделали 3 категории в зависимости от ранга, сделали dummy-переменные
- also_view - удалили, но наверное можно было бы использовать
- main_cat - сделали dummy-переменные
- price - сделали 3 категории в зависимости от цены, сделали dummy-переменные
- also_buy - удалили, но наверное можно было бы использовать
В итоге получили большое количество признаков, которые можем использовать в нашей модели.

# Обучение простой модели

In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
# Разделим тренировочный датасет на тренировочную и тестовую выборки
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)

ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'],)))

In [ ]:
NUM_THREADS = 4  # число потоков
NUM_COMPONENTS = 30  # число параметров вектора
NUM_EPOCHS = 20  # число эпох обучения

model_b = LightFM(learning_rate=0.1, loss='logistic',
                  no_components=NUM_COMPONENTS)
model_b = model_b.fit(ratings_coo, epochs=NUM_EPOCHS,
                      num_threads=NUM_THREADS)

preds_b = model_b.predict(test_data.userid.values,
                          test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_b)

In [ ]:
# Подбор параметров модели

"""roc_dict = {}
RANDOM_STATE = 20
i = 0
for lr in [0.112]:  # np.arange(0.098, 0.151, 0.001):
    for lf in ['logistic']:
        for ls in ['adagrad']:
            for nc in [170]:
                for ne in [10, 11, 12, 13, 14, 15, 16, 17, 18]:
                    for nt in [12]:
                        param = {}
                        param = {'learning_rate': lr,
                                 'loss': lf,
                                 'learning_schedule': ls,
                                 'no_components': nc,
                                 'epochs': ne,
                                 'num_threads': nt
                                 }
                        model_ = LightFM(
                            learning_rate=lr,
                            loss=lf,
                            no_components=nc,
                            learning_schedule=ls,
                            random_state=RANDOM_STATE
                        ).fit(
                            ratings_coo,
                            epochs=ne,
                            num_threads=nt
                        )
                        preds_ = model_.predict(test_data.userid.values,
                                                test_data.itemid.values)
                        roc = sklearn.metrics.roc_auc_score(
                            test_data.rating, preds_)
                        roc_dict[roc] = param
                        print(i, roc, param)
                        i += 1

best_result = max(roc_dict.keys())
best_param = roc_dict[best_result]
print(best_result, best_param)"""

Некоторые промежуточные результаты

(0.7512193985520158, {'learning_rate': 0.08, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 25, 'num_threads': 12}),

(0.7514235651698367 {'learning_rate': 0.08, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 24, 'num_threads': 12}),

(0.7514739213753435 {'learning_rate': 0.083, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 24, 'num_threads': 12}),

(0.7515141228065838 {'learning_rate': 0.084, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 24, 'num_threads': 12}),

(0.7519620719322733 {'learning_rate': 0.084, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 21, 'num_threads': 12})

0.752250319248385 {'learning_rate': 0.089, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 21, 'num_threads': 12}

0.7523649127274752 {'learning_rate': 0.089, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 19, 'num_threads': 12}

0.752754427151223 {'learning_rate': 0.094, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 19, 'num_threads': 12}

0.752912859782567 {'learning_rate': 0.1, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 19, 'num_threads': 12}

0.7530235217587499 {'learning_rate': 0.1, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 17, 'num_threads': 12}

0.7534181653143477 {'learning_rate': 0.112, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 17, 'num_threads': 12}

0.7536314887244534 {'learning_rate': 0.112, 'loss': 'logistic', 'learning_schedule': 'adagrad', 'no_components': 170, 'epochs': 16, 'num_threads': 12}

In [ ]:
# Запустим модель с подобранными параметрами

LR = 0.112
NUM_THREADS = 12 #число потоков
NUM_COMPONENTS = 170 #число параметров вектора 
NUM_EPOCHS = 16 #число эпох обучения
LEARNING_SCHEDULE = 'adagrad'
LOSS_FUNCTION = 'logistic'
RANDOM_STATE = 20

model_p = LightFM(
    learning_rate=LR,
    loss=LOSS_FUNCTION,
    no_components=NUM_COMPONENTS,
    learning_schedule = LEARNING_SCHEDULE,
    random_state = RANDOM_STATE
).fit(
    ratings_coo,
    epochs=NUM_EPOCHS,
    num_threads=NUM_THREADS
)

In [ ]:
preds_p = model_p.predict(test_data.userid.values,
                          test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating,preds_p)

In [ ]:
preds_pt = model_p.predict(test.userid.values,
                          test.itemid.values)

normalized_preds_p = (preds_pt - preds_pt.min())/(preds_pt - preds_pt.min()).max()

submission['rating'] = normalized_preds_p
submission.to_csv(path + '/data/submission_param_1.csv', index=False)

In [ ]:
# Сохраним модель
import pickle

with open(path + '/LightFM_best_param.pickle', 'wb') as fle:
    pickle.dump(model_p, fle, protocol=pickle.HIGHEST_PROTOCOL)

# Добавляем item_features и user_features

Рассмотрим возможность добавления item_features и user_features в модель LightFM при помощи lightfm.dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(path + '/data/extended_df.csv')

In [ ]:
df.head()

In [ ]:
# Можно уберать часть фич, так как в инстансе на Kaggle не хватает памяти. 
# Для обработки всех фич из датасета использовался комп с 32ГБ оперативки и двумя процессорами по 6 ядер
for key in df.keys():
    #print(key)
    if key[:8]=='main_cat':
        df.drop(key, axis=1, inplace=True)
    if key[:8]=='category':
        df.drop(key, axis=1, inplace=True)
    if key[:5]=='brand':
        df.drop(key, axis=1, inplace=True)
    if key[:11]=='description':
        df.drop(key, axis=1, inplace=True)

In [ ]:
df.drop_duplicates()

In [ ]:
test_ = df[df['is_test']==1].drop(['is_test'], axis=1)
train_ = df[df['is_test']==0].drop(['is_test'], axis=1)

features_user_train = train_[['userid', 'verified']]
features_item_train = train_.drop(columns=['userid','verified','rating'])
df1 = train_[['userid','itemid','rating']]

features_user_test = test_[['userid', 'verified']]
features_item_test = test_.drop(columns=['userid','verified','rating'])

In [ ]:
features_user = pd.concat([features_user_train, features_user_test], ignore_index=True)
features_item = pd.concat([features_item_train, features_item_test], ignore_index=True)

Нам нужно вызвать метод fit, чтобы сообщить LightFM id пользователей, id продуктов, и дополнительные фичи пользователя или продукта.

Мы передадим методу fit три параметра:

* users: список всех пользователей
* items: список всех продуктов
* item_features: список дополнительных фичей продукта

In [ ]:
item_f = []
col = []
unique_f1 = []
for column in features_item.drop(['itemid'], axis=1):
    col += [column]*len(features_item[column].unique())
    unique_f1 += list(features_item[column].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    item_f.append(res)
    #print(res)

In [ ]:
user_f = []
col = []
unique_f1 = []
for column in features_user.drop(['userid'], axis=1):
    col += [column]*len(features_user[column].unique())
    unique_f1 += list(features_user[column].unique())
for x,y in zip(col, unique_f1):
    res = str(x)+ ":" +str(y)
    user_f.append(res)
    print(res)

In [ ]:
# Вызовем метод fit для нашего датасета

from lightfm.data import Dataset
# we call fit to supply userid, item id and user/item features
dataset1 = Dataset()
dataset1.fit(
        df['userid'].unique(), # all the users
        df['itemid'].unique(), # all the items
        user_features = user_f,
        item_features = item_f
)

Теперь, когда у нас есть готовый скелет датасета, мы готовы добавить в него фактические взаимодействия (interactions) и оценки (ratings).

## Building interactions - построение взаимодействий

In [ ]:
# plugging in the interactions and their weights
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in df1.values ])

матрица взаимодействий сообщает нам, взаимодействовал ли вообще пользователь с элементом, а матрица весов дает количественную оценку этого конкретного взаимодействия.

In [ ]:
#interactions.todense()

In [ ]:
#weights.todense()

## Строим Item Features

In [ ]:
ll = []
for column in features_item.drop(['itemid'], axis=1):
    ll.append(column + ':')
#print(ll)

In [ ]:
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,0,'del'],
    resultant output = ['f1:1', 'f2:1', 'f3:0', 'loc:del']

    """
    result = []
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [ ]:
ad_subset = features_item.drop(['itemid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
item_feature_list = []
for item in ad_list:
    item_feature_list.append(feature_colon_value(item))
#print(f'Final output: {item_feature_list[0:5]}')

Наконец, мы должны связать каждый элемент item_feature_list с соответствующими идентификаторами продуктов.

In [ ]:
item_tuple = list(zip(features_item.itemid, item_feature_list))
#item_tuple[0:5]

In [ ]:
# Мы получили желаемый вид для ввода данных для метода build_item_features. Вызовем этот метод
item_features = dataset1.build_item_features(item_tuple, normalize= False)
#item_features.todense()

В приведенной выше матрице item_features строки - это продукты, а столбцы - это фичи продуктов. 1 присутствует всякий раз, когда у этого продукта есть эта конкретная фича, присутствующая в тренировочном датасете.

## Создадим user_features

In [ ]:
# Аналогично item_features
ll = []
for column in features_user.drop(['userid'], axis=1):
    ll.append(column + ':')
print(ll)

In [ ]:
ad_subset = features_user.drop(['userid'], axis=1)
ad_list = [x.tolist() for x in ad_subset.values]
user_feature_list = []
for user in ad_list:
    user_feature_list.append(feature_colon_value(user))
print(f'Final output: {user_feature_list[0:5]}')

In [ ]:
user_tuple = list(zip(features_user.userid, user_feature_list))

In [ ]:
user_features = dataset1.build_user_features(user_tuple, normalize= False)
#user_features.todense()

## Обучаем модель

In [ ]:
# Создадим словари, с помощью которых по id в датасете LightFM мы сможем находить id в датасете df.

user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()

In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
model_f = LightFM(learning_rate=0.112,
    loss='logistic',
    no_components=170,
    learning_schedule='adagrad',
    random_state=20)
model_f.fit(interactions, # spase matrix representing whether user u and item i interacted
    user_features = user_features,
    item_features = item_features, # we have built the sparse matrix above
    sample_weight = weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
    epochs=16,
    num_threads=12)

In [ ]:
# Получим значение AUC

train_auc = auc_score(model_f,
                      interactions,
                      user_features = user_features,
                      item_features=item_features
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

## Предсказания

Метод predict принимает три параметра на вход:

* мэппинги (отображения) id пользователей (например: для получения прогнозов для первого пользователя необходимо передать 0; для второго 1 и т. д.). Эти мэппинги доступны из словаря user_id_map.
* список id продуктов (опять же не itemid из датасета df, а мэппинги (отображение), которые доступны из item_id_map), для которых вы хотите получить рекомендации.
* item_features

Предскажем пока на известных данных тренировочного датасета

In [ ]:
user_ids = df1.userid.apply(lambda x: user_id_map[x])
item_ids = df1.itemid.apply(lambda x: item_id_map[x])
preds_f = model_f.predict(user_ids.values, item_ids.values, user_features=user_features, item_features=item_features)

sklearn.metrics.roc_auc_score(df1.rating,preds_f)

Logistic: 0.5414955799065918

Видим, что добавление фичей негативно сказывается на метрике roc_auc. Вполне ожидемо, что модель с оптимальными параметрами дает лучший результат

# Получение рекомендаций

In [ ]:
# Достаём эбмеддинги
item_biases, item_embeddings = model_f.get_item_representations(features=item_features)

In [ ]:
# Сохраняем полученные эмбеддинги
import pickle

with open('item_embeddings_rec.pickle', 'wb') as fle:
    pickle.dump(item_embeddings, fle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
#!pip install nmslib -q

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import nmslib
 
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

## Предсказания для новых пользователей/продуктов
Именно поэтому мы в первую очередь создавали гибридную рекомендательную систему. Для нового пользователя это то, что мы знаем - у него есть значения для feature1, feature2, feature3 как 1,1 и 0 соответственно. Кроме того, verified = 1.

item_feature_list = ['feature: 1', 'feature2: 1', 'feature3: 0', 'verified: 1']

Теперь мы не можем передать это напрямую методу predict. Мы должны преобразовать этот формат в вид, понятный нашей модели lightFM. В идеале входные данные должны выглядеть как одна из строк в матрице item_features.

Функции ниже преобразует item_feature_list и user_feature_list в требуемый формат.

In [ ]:
item_feature_list = ['main_cat:Other']
user_feature_list = ['verified:1']

In [ ]:
from scipy import sparse

def format_newitem_input(item_feature_map, item_feature_list): 
    num_features = len(item_feature_list)
    normalised_val = 1.0 
    target_indices = []
    for feature in item_feature_list:
        try:
            target_indices.append(item_feature_map[feature])
        except KeyError:
            print("new item feature encountered '{}'".format(feature))
            pass

    new_item_features = np.zeros(len(item_feature_map.keys()))
    for i in target_indices:
        new_item_features[i] = normalised_val
    new_item_features = sparse.csr_matrix(new_item_features)
    return(new_item_features)

def format_newuser_input(user_feature_map, user_feature_list):
    num_features = len(user_feature_list)
    normalised_val = 1.0 
    target_indices = []
    for feature in user_feature_list:
        try:
            target_indices.append(user_feature_map[feature])
        except KeyError:
            print("new user feature encountered '{}'".format(feature))
            pass

    new_user_features = np.zeros(len(user_feature_map.keys()))
    for i in target_indices:
        new_user_features[i] = normalised_val
    new_user_features = sparse.csr_matrix(new_user_features)
    return(new_user_features)

In [ ]:
import numpy as np
# Наконец, мы можем сделать предсказания для нового пользователя:

new_user_features = format_newuser_input(user_feature_map, user_feature_list)
preds_nu = model_f.predict(0, item_ids.values, user_features=new_user_features, item_features=item_features)
preds_nu

Здесь первый аргумент, то есть 0, больше не относится к отображаемому идентификатору для первого продукта в датасете. Вместо этого это означает - выберите первую строку разреженной матрицы new_item_features. Передача любого значения, отличного от 0, вызовет ошибку, и это правильно, поскольку в new_item_features нет строк кроме первой строки row0.

In [ ]:
# Аналогично для нового продукта

new_item_features = format_newitem_input(item_feature_map, item_feature_list)
preds_np = model_f.predict(user_ids.values, len(user_ids.values)*[0], user_features=user_features, item_features=new_item_features)

In [ ]:
# И в случае нового пользователя и нового продукта

preds_nup = model_f.predict(0, [0], user_features=new_user_features, item_features=new_item_features)

## Заключение
Таким образом, с помощью добавления item_features и user_features, появляется возможность обойти проблему холодного старта. Мы можем получить информацию о пользователе, например, при регистрации, и использовать эти данные для получения рекомендаций.

# Рекомендательная система с использованием collab learner от fast.ai

Потренируемся работать с библиотекой грубокого обучения fast.ai. Перед запуском ноутбука рекомендуется подключить gpu для увеличения скорости обучения.

In [ ]:
from fastai.tabular import *
from fastai.collab import *
import pandas as pd

# Загружаем датасеты
data_train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
data_train = data_train[['userid','itemid','rating']]
data_train.columns = ['user_id','item_id','rating']

# Удалим дубликаты из тренировочного датасета
data_train.drop_duplicates(inplace = True)

test_data = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
test_data = test_data[['userid', 'itemid']]
test_data.columns = ['user_id','item_id']

submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

In [ ]:
data = CollabDataBunch.from_df(
    data_train,
    seed=42,
    user_name='user_id',
    item_name='item_id',
    rating_name='rating',
    )
data.show_batch()

## Модель EmbeddingDotBias

In [ ]:
learn = collab_learner(data, n_factors=50, y_range=(0, 1), wd=1e-2)

In [ ]:
learn.lr_find() # find learning rate
learn.recorder.plot() # plot learning rate graph

In [ ]:
learn.fit_one_cycle(5, 1e-2)

In [ ]:
learn.save("dot_bias_model", return_path=True)

In [ ]:
# Укажем тестовый датасет

data_collab = CollabDataBunch.from_df(data_train, test=test_data, seed=42, valid_pct=0.2, 
                                      user_name='user_id', item_name='item_id', rating_name='rating')
learn = collab_learner(data_collab, n_factors=50, y_range=(0, 1), wd=1e-2)

In [ ]:
# Загрузим ранее сохраненную модель

learn_loaded = learn.load(Path('dot_bias_model'))

In [ ]:
# Получим предсказания

preds, y = learn_loaded.get_preds(DatasetType.Test)

In [ ]:
submission['rating']= preds
submission.to_csv('fastai_submission.csv', index=False)

На Kaggle получен результат 0.73794, что примерно соответствует базовой модели. Возможно улучшение результата путем подбора параметров или применения нейросети.

# Основные итоги

В процессе работы над проектом удалось реализовать почти все поставленные задачи. Был произведен детальный анализ дополнительных данных, часть которых была добавлена в модель, что позволило немного улучшить результаты.

Выполнен разведывательный анализ данных, произведена генерация новых признаков, на рабочем компьютере произведен расчет модели со всеми признаками (к сожалению, ноутбук на Kaggle, в моем случае, тянул не больше 20 признаков).

Наилучшие результаты по метрике ROC-AUC были достигнуты для модели без дополнительных признаков, только путем подбора гипер-параметров этой модели.

Попробовал поработать с библиотекой fastai, впечатления пока противоречивые. Требуется более детальная проработка.

На ноутбуке Kaggle не удалось использовать библиотеку nmslib, поэтому реализация функцонала по рекомендациям только на рабочем компьютере и github.


Пример, приведенный в модуле не работает. Не проходит деплой на Heroku не может скомпилировать библиотеки. Пробовал и с домашнего компьютера и через github.
Сам проект запускается в обоих случаях.
streamlit run https://github.com/samakarov/hiroku_deploy/blob/main/app.py